In [ ]:
import sys
sys.path.append("../")

In [ ]:
header_length = len("2022-10-05 11:56:15.580  INFO 8982 --- [           main] ")
header_length

In [ ]:
event_length = len("f.i.sirene4.repertoire.BatchApplication  ")
event_length

In [ ]:
first_line = "2022-10-05 11:56:15.580  INFO 8982 --- [           main] f.i.sirene4.repertoire.BatchApplication  : Starting BatchApplication v2.3.3 using Java 11.0.16.1 on qfbatrelst01.ad.insee.intra with PID 8982 (/opt/insee/sirene4/qf3/lib/repertoire-batch-2.3.3.jar started by www-data in /opt/insee/sirene4/qf3/tmp)"

In [ ]:
line_without_timestamp = first_line[header_length:]
event_type = line_without_timestamp[:event_length]
event_type.rstrip()

In [ ]:
description = line_without_timestamp[event_length + 2:]
description

In [ ]:
import pandas as pd

def extract_log_info(f):
    event_types = []
    descriptions = []
    for line in f:
        line_without_timestamp = line[header_length:]
        if not line_without_timestamp:
            continue
        event_types.append(line_without_timestamp[:event_length].rstrip())
        descriptions.append(line_without_timestamp[event_length + 2:])
    return pd.DataFrame(list(zip(event_types, descriptions)), columns =['event_type', 'description'])

with open("../data/api_log.log") as f:
    df = extract_log_info(f)

In [ ]:
df

In [ ]:
identifier = "s.i.AbstractBatchCodificationServiceImpl"
log_info = "r.b.j.c.s.i.BatchCodificationServiceImpl"
raw_input = "stractLiasse1ToLiasseVarInteretProcessor"

In [ ]:
df_ids = df[df.event_type == identifier]
df_ids.head()

In [ ]:
df_info = df[df.event_type == log_info]
df_info.head()

In [ ]:
df_ids.shape

In [ ]:
df_info.shape

In [ ]:
df_raw_input = df[df.event_type == raw_input]
df_raw_input.head()

In [ ]:
df_raw_input = df_raw_input[df_raw_input.description.str.startswith("LiasseVarInteretCodification")]
df_raw_input.head()

In [ ]:
df_raw_input.shape

In [ ]:
test_str = df_raw_input.iloc[0, 1]
test_str

In [ ]:
import re

regex = re.compile(r'norme=([^,]*),')
matches = regex.search(test_str)
matches.group(1)

In [ ]:
regex = re.compile(r'liasseType=([^,]*),')
matches = regex.search(test_str)
matches.group(1)

In [ ]:
regex

In [ ]:
raw_fields = [
    "norme",
    "siren",
    "nic",
    "liasseType",
    "categorieJuridique",
    "domas",
    "ssdom",
    "domaineAssoc",
    "ssDomaineAssoc",
    "libelleActivitePrincipaleEtablissement",
    "sedentarite",
    "natureActivites",
    "surface",
    "lieuExercice",
    "presenceSalaries"
]
raw_regexes = [re.compile(r'{}'.format(field + '=([^,]*)[,\]]')) for field in raw_fields]
raw_regexes

In [ ]:
def parse_raw_input(raw_input, fields, regexes):
    raw_input_dict = {}
    for field, regex in zip(fields, regexes):
        matches = regex.search(raw_input)
        raw_input_dict[field] = matches.group(1)
    return raw_input_dict

In [ ]:
parse_raw_input(test_str, raw_fields, raw_regexes)

In [ ]:
dict_series = [parse_raw_input(raw_input, raw_fields, raw_regexes) for raw_input in df_raw_input.description]

In [ ]:
pd.DataFrame(list(dict_series))

In [ ]:
test_str = df_info.iloc[0, 1]
test_str

In [ ]:
info_fields = [
    "libelleActivite",
    "natureActivites",
    "liasseType",
    "evenementType",
    "surface",
    "libelleNettoye",
    "predictions",
    "bilan"
]
info_regexes = [re.compile(r'{}'.format(field + '=([^,]*),')) for field in info_fields]

info_fields.append("fasttextVersion")
info_regexes.append(re.compile(r'fasttextVersion=([^,]*)\]'))

info_regexes

In [ ]:
parse_raw_input(test_str, info_fields, info_regexes)

In [ ]:
def extract_first_pred(predictions):
    regex = re.compile(r'proposé = ([^;]*) ;.*associée = ([^;]*)\].*proposé = ([^;]*) ;.*associée = ([^;]*)\]')
    matches = regex.search(predictions)
    first_code = matches.group(1)
    second_code = matches.group(3)
    first_proba = matches.group(2)
    second_proba = matches.group(4)
    return (first_code, second_code, float(first_proba), float(second_proba))

extract_first_pred(parse_raw_input(test_str, info_fields, info_regexes)["predictions"])

In [ ]:
dict_series = [parse_raw_input(info_input, info_fields, info_regexes) for info_input in df_info.description]

In [ ]:
df = pd.DataFrame(list(dict_series))

In [ ]:
predictions = [extract_first_pred(predictions) for predictions in df["predictions"]]
df["first_pred"] = [prediction[0] for prediction in predictions]
df["second_pred"] = [prediction[0] for prediction in predictions]
df["first_proba"] = [prediction[0] for prediction in predictions]
df["second_proba"] = [prediction[0] for prediction in predictions]

df

In [ ]:
import pandas as pd

df = pd.read_csv("comparison.csv")

In [ ]:
df = df[~df.RAW.str.contains("oeuvre")]

In [ ]:
df.RAW.iloc[7]

In [ ]:
df.RAW.replace(
    to_replace=r"(\s|^)([a-z]{1})(?:\s|$)", value=r'\1', regex=True
).iloc[7]

In [ ]:
df.RAW.replace(
    to_replace=r"\b([a-z]{1})\b", value=' ', regex=True
).iloc[7]

In [ ]:
x = compare_libs[~compare_libs["CHECK"]]

1) Prepro mots vide de sens
2) ```"œ"``` pas décodé de la même manière
3) ```"€"``` pas décodé de la même manière
4) ```"Yeti"``` pas racinisé de la même manière
5) ```"Terassemment"``` pas racinisé de la même manière

In [ ]:
model = fasttext.load_model("../models/model.bin")

In [ ]:
res = model.predict(df[compare_libs.CHECK].libelleNettoye.to_list(), k=2)

first_pred = [res[0][i][0].replace("__label__", "") for i in range(len(res[0]))]
second_pred = [res[0][i][1].replace("__label__", "") for i in range(len(res[0]))]
first_proba = [res[1][i][0] for i in range(len(res[1]))]
second_proba = [res[1][i][1] for i in range(len(res[1]))]

In [ ]:
lib_clean = df[compare_libs.CHECK].libelleNettoye.apply(lambda x : x.split(" AUTO")[0]).to_list()

df_compare = pd.DataFrame({
    "Lib" : df[compare_libs.CHECK].libelleNettoye, 
    "pred_1_PY" : first_pred, 
    "pred_1_JAVA" : df[compare_libs.CHECK].first_pred,
    "pred_2_PY" : second_pred, 
    "pred_2_JAVA" : df[compare_libs.CHECK].second_pred,
    "prob_1_PY" : first_proba, 
    "prob_1_JAVA" : df[compare_libs.CHECK].first_proba,
    "prob_2_PY" : second_proba, 
    "prob_2_JAVA" : df[compare_libs.CHECK].second_proba
    })


In [ ]:
df_compare[df_compare.pred_1_PY != df_compare.pred_1_JAVA]

In [ ]:
df_compare[df_compare.pred_2_PY != df_compare.pred_2_JAVA]

In [ ]:
df_compare[(df_compare.pred_1_PY != df_compare.pred_1_JAVA) | (df_compare.pred_2_PY != df_compare.pred_2_JAVA)]